# Practice Session 08: Data streams



Author: <font color="blue">Leticia Martin Cabrera</font>

E-mail: <font color="blue">leticia.martin03@estudiant.upf.edu</font>

Date: <font color="blue">20/11/2021</font>

In [35]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re

In [36]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\letic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 0. Dataset and how to iterate

In [37]:
# Leave this code as-is

INPUT_FILE = "DCEP-reports-en.txt.gz"

In [38]:
# Leave this code as-is

# Producer in Python that reads a filename by words
def read_by_words(filename, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Regular expression to identify words having 4 letters or more and beginning with a-z
        word_expr = re.compile('^[a-z][a-z0-9]{4,}$', re.IGNORECASE)

        # Iterate through lines in the file
        for line in file:
            
            if counter > max_words and max_words != -1:
                break
                
            for word in nltk.word_tokenize(line):
                if word_expr.match(word):
                    counter += 1
                    
                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word
                    yield word

In [39]:
# Leave this code as-is

# Iterate through the file
for word in read_by_words(INPUT_FILE, max_words=300000, report_every=100000):
    # Prints 0.005% of words
    if random.random() < 0.0001:
        print("Current word '%s'" % (word)) 

Current word 'implemented'
Current word 'difficulties'
Current word 'research'
Current word 'States'
Current word 'producers'
Current word 'opinion'
Current word 'funds'
Current word 'approach'
- Read 100000/300000 words so far
Current word 'allergies'
Current word 'sixth'
Current word 'study'
Current word 'Rights'
Current word 'concerned'
Current word 'announced'
Current word 'respect'
Current word 'available'
Current word 'ability'
Current word 'Union'
Current word 'Members'
Current word 'Total'
Current word 'films'
Current word 'awareness'
- Read 200000/300000 words so far
Current word 'determine'
Current word 'poverty'
Current word 'gender'
Current word 'recourse'
Current word 'products'
Current word 'deposit'
Current word 'guidelines'
Current word 'shall'
Current word 'necessary'
Current word 'instrument'
Current word 'Commission'
Current word 'comitology'
Current word 'together'
- Read 300000/300000 words so far


# 1. Determine approximately the top-5 words

In [40]:
#Implementing function add_to_reservoir  that adds an item to the reservoir, maintaining its size
def add_to_reservoir(reservoir, item, max_reservoir_size):
    
    #Checking if reservoir is full
    if len(reservoir) >= max_reservoir_size:
        
        #Discard random elemnt and insert the new item in the same position 
        random_pos = random.randint(0,max_reservoir_size-1)
        reservoir[random_pos]  = item
        
    #Adding new item without deleting any element    
    else:
        reservoir.append(item)
        
    assert(len(reservoir) <= max_reservoir_size)

In [41]:
#Creating reservoir_sampling funtion that iterates through the file using the reservoir sampling method
def reservoir_sampling(filename, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []

    words_read = 0  #word counter

    for word in read_by_words(filename, max_words=max_words, report_every=report_every):
        
        words_read += 1 #increases the word counter

        #Checking if reservoir is full
        if words_read >= reservoir_size:
            random_element = random.random()
            
            
            #checking if the new element arrives with problability s/n (reservoir_size/words_read)
            if random_element <= reservoir_size/words_read:
                
                #inserting the new word into reservoir
                add_to_reservoir(reservoir, word, reservoir_size)
                
        #Adding new word without deleting any word       
        else: 
            add_to_reservoir(reservoir, word, reservoir_size)
            

    return (words_read, reservoir)

In [42]:
# Testing the function reservoir_sampling

reservoir_size = 500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=300000, report_every=100000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 100000/300000 words so far
- Read 200000/300000 words so far
- Read 300000/300000 words so far
Number of items seen    : 300007
Number of items sampled : 500


In [43]:
#compute the absolute frequencies of the top 5

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

9 Article
7 Parliament
7 European
7 Amendment
6 their
5 which
5 shall
4 including
4 development
4 States


In [44]:
#print the top items and their relative frequencies
freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)/len(reservoir)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for relative_frequency, word in most_frequent_items:
    print("%.3f %s" % (relative_frequency, word))

0.018 Article
0.014 Parliament
0.014 European
0.014 Amendment
0.012 their
0.010 which
0.010 shall
0.008 including
0.008 development
0.008 States


In [45]:
#Print the relative and absolute frequency of the words increasing the max limit of words

#Reservoir sizes
sizes = [100,500,1000,3000,5000]

for size in sizes:
    print("\n############### Reservoir size %d ###############" %size)
    
    #Fixing max limit of words at one million
    (items_seen, reservoir) = reservoir_sampling(INPUT_FILE, size, max_words=1000000, report_every=250000)
    
    #Compute absolute and relatives frequencies
    freq = {}
    for item in reservoir:
        freq[item] = reservoir.count(item)

    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]
    
    #printing the results 
    for absolute_frequency, word in most_frequent_items:
        print("%s --> Absolute frequency: %d |Relative frequency: %.4f" % (word, absolute_frequency*items_seen/len(reservoir), absolute_frequency/len(reservoir)))



############### Reservoir size 100 ###############
- Read 250000/1000000 words so far
- Read 500000/1000000 words so far
- Read 750000/1000000 words so far
- Read 1000000/1000000 words so far
terms --> Absolute frequency: 20000 |Relative frequency: 0.0200
rights --> Absolute frequency: 20000 |Relative frequency: 0.0200
Parliament --> Absolute frequency: 20000 |Relative frequency: 0.0200
would --> Absolute frequency: 10000 |Relative frequency: 0.0100
workers --> Absolute frequency: 10000 |Relative frequency: 0.0100

############### Reservoir size 500 ###############
- Read 250000/1000000 words so far
- Read 500000/1000000 words so far
- Read 750000/1000000 words so far
- Read 1000000/1000000 words so far
information --> Absolute frequency: 12000 |Relative frequency: 0.0120
Commission --> Absolute frequency: 12000 |Relative frequency: 0.0120
which --> Absolute frequency: 10000 |Relative frequency: 0.0100
protection --> Absolute frequency: 10000 |Relative frequency: 0.0100
national --> A

We have not obtained stable results with any of the reservoir size we have tested. We can see that at best only one word in the top 3 matches. 

In [46]:
#Print the relative and absolute frequency of the words without max limit of words

#Reservoir sizes
sizes = [100,500,1000,3000,5000]

for size in sizes:
    print("\n############### Reservoir size %d ###############" %size)
    
    #Computing reservoir sampling algortihm without max limit of words
    (items_seen, reservoir) = reservoir_sampling(INPUT_FILE, size)
    
    #Compute absolute and relatives frequencies
    freq = {}
    for item in reservoir:
        freq[item] = reservoir.count(item)

    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]
    
    #printing the results
    for absolute_frequency, word in most_frequent_items:
        print("%s --> Absolute frequency: %d |Relative frequency: %.4f" % (word, absolute_frequency*items_seen/len(reservoir), absolute_frequency/len(reservoir)))


############### Reservoir size 100 ###############
Justification --> Absolute frequency: 308470 |Relative frequency: 0.0300
which --> Absolute frequency: 205647 |Relative frequency: 0.0200
their --> Absolute frequency: 205647 |Relative frequency: 0.0200
strategy --> Absolute frequency: 205647 |Relative frequency: 0.0200
regional --> Absolute frequency: 205647 |Relative frequency: 0.0200

############### Reservoir size 500 ###############
should --> Absolute frequency: 143952 |Relative frequency: 0.0140
Commission --> Absolute frequency: 143952 |Relative frequency: 0.0140
shall --> Absolute frequency: 123388 |Relative frequency: 0.0120
Amendment --> Absolute frequency: 123388 |Relative frequency: 0.0120
proposed --> Absolute frequency: 102823 |Relative frequency: 0.0100

############### Reservoir size 1000 ###############
European --> Absolute frequency: 133670 |Relative frequency: 0.0130
Member --> Absolute frequency: 123388 |Relative frequency: 0.0120
Commission --> Absolute frequenc

We can observe that if we increase the size of reservoir there are more words matches. Commission and European are the most frequent words. However, we can not conclude that  assure the stability of the output. I think that we should choose a higher size but the inconvenience of this is that the code take more time. 

# 2. Determine approximately the distinct number of words

In [47]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [48]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

In [49]:
#Estimating the number of distinc words using Flajolet-Martin probabilistic counting method
number_of_passes = 10
all_estimates = []

#Executing 3 times the algorithm 
for i in range(3):
    estimates = []
    print("\n Test %d" % (i+1))
    for i in range(number_of_passes):
        # read the file and generate an estimate
        R = 0

        #Create hash funcion h
        h = random_hash_function()

        #reading the entire file
        for word in read_by_words(INPUT_FILE, max_words = 30000):

            #Compute hash value 
            hash_value = h(word)

            #calculate the number of trailing zeroes in h(u)
            trailing_zeros = count_trailing_zeroes(hash_value)

            #Maintain R as the maximum value of trailing_zeros seen so far
            if trailing_zeros > R:
                R = trailing_zeros

        #Add 2^R as an estimate for the number of distinct words seen       
        estimate = 2**R
        estimates.append(estimate)

        print("Estimate on pass %d: %d distinct words" % (i+1, estimate))
    all_estimates.append(estimates)



 Test 1
Estimate on pass 1: 2048 distinct words
Estimate on pass 2: 16384 distinct words
Estimate on pass 3: 8192 distinct words
Estimate on pass 4: 2048 distinct words
Estimate on pass 5: 1024 distinct words
Estimate on pass 6: 16384 distinct words
Estimate on pass 7: 2048 distinct words
Estimate on pass 8: 2048 distinct words
Estimate on pass 9: 16384 distinct words
Estimate on pass 10: 2048 distinct words

 Test 2
Estimate on pass 1: 2048 distinct words
Estimate on pass 2: 4096 distinct words
Estimate on pass 3: 4096 distinct words
Estimate on pass 4: 1024 distinct words
Estimate on pass 5: 4096 distinct words
Estimate on pass 6: 4096 distinct words
Estimate on pass 7: 8192 distinct words
Estimate on pass 8: 8192 distinct words
Estimate on pass 9: 1024 distinct words
Estimate on pass 10: 2048 distinct words

 Test 3
Estimate on pass 1: 1024 distinct words
Estimate on pass 2: 4096 distinct words
Estimate on pass 3: 8192 distinct words
Estimate on pass 4: 2048 distinct words
Estimate

In [50]:
# Compute the median of estimates obtained in 3 separate runs 
for i,estimates in enumerate(all_estimates):
    print("TEST %d"% (i+1))
    print("* Average of estimate: %.1f" % (statistics.mean(estimates)))
    print("* Median  of estimates: %.1f" % (statistics.median(estimates)))

TEST 1
* Average of estimate: 6860.8
* Median  of estimates: 2048.0
TEST 2
* Average of estimate: 3891.2
* Median  of estimates: 4096.0
TEST 3
* Average of estimate: 4300.8
* Median  of estimates: 4096.0


In [51]:
#Estimating the number of distinc words using Flajolet-Martin probabilistic counting method
#Increase the number of passes
number_of_passes = 20
all_estimates = []

#Executing 3 times the algorithm 
for i in range(3):
    estimates = []
    print("\n Test %d" % (i+1))
    for i in range(number_of_passes):
        # read the file and generate an estimate
        R = 0

        #Create hash funcion h
        h = random_hash_function()

        #reading the entire file
        for word in read_by_words(INPUT_FILE, max_words = 30000):

            #Compute hash value 
            hash_value = h(word)

            #calculate the number of trailing zeroes in h(u)
            trailing_zeros = count_trailing_zeroes(hash_value)

            #Maintain R as the maximum value of trailing_zeros seen so far
            if trailing_zeros > R:
                R = trailing_zeros

        #Add 2^R as an estimate for the number of distinct words seen       
        estimate = 2**R
        estimates.append(estimate)

        print("Estimate on pass %d: %d distinct words" % (i+1, estimate))
    all_estimates.append(estimates)




 Test 1
Estimate on pass 1: 16384 distinct words
Estimate on pass 2: 4096 distinct words
Estimate on pass 3: 4096 distinct words
Estimate on pass 4: 32768 distinct words
Estimate on pass 5: 4096 distinct words
Estimate on pass 6: 4096 distinct words
Estimate on pass 7: 4096 distinct words
Estimate on pass 8: 2048 distinct words
Estimate on pass 9: 262144 distinct words
Estimate on pass 10: 4096 distinct words
Estimate on pass 11: 4096 distinct words
Estimate on pass 12: 4096 distinct words
Estimate on pass 13: 8192 distinct words
Estimate on pass 14: 4096 distinct words
Estimate on pass 15: 16384 distinct words
Estimate on pass 16: 1024 distinct words
Estimate on pass 17: 4096 distinct words
Estimate on pass 18: 4096 distinct words
Estimate on pass 19: 4096 distinct words
Estimate on pass 20: 16384 distinct words

 Test 2
Estimate on pass 1: 2048 distinct words
Estimate on pass 2: 4096 distinct words
Estimate on pass 3: 2048 distinct words
Estimate on pass 4: 512 distinct words
Estima

In [52]:
## Compute the median of estimates obtained in 3 separate runs 
for i,estimates in enumerate(all_estimates):
    print("TEST %d"% (i+1))
    print("* Average of estimate: %.1f" % (statistics.mean(estimates)))
    print("* Median  of estimates: %.1f" % (statistics.median(estimates)))

TEST 1
* Average of estimate: 20224.0
* Median  of estimates: 4096.0
TEST 2
* Average of estimate: 33382.4
* Median  of estimates: 4096.0
TEST 3
* Average of estimate: 18585.6
* Median  of estimates: 6144.0


<font size="+1" color="red">Remove the limit of max words, or set to a high number, but notice that you do no need to use more than one hour of computer processing time, and perform the 10 passes. Replace this cell with the results you obtained in each pass, and whether the average or the median seem more appropriate for this probabilistic counting.</font>

In [53]:
#Estimating the number of distinc words using Flajolet-Martin probabilistic counting method removing the limit of max words.

number_of_passes = 10
estimates = []

for i in range(number_of_passes):
    #read the file and generate an estimate
    R = 0

    #Create hash funcion h
    h = random_hash_function()

    #reading the entire file
    for word in read_by_words(INPUT_FILE):

        #Compute hash value 
        hash_value = h(word)

        #calculate the number of trailing zeroes in h(u)
        trailing_zeros = count_trailing_zeroes(hash_value)

        #Maintain R as the maximum value of trailing_zeros seen so far
        if trailing_zeros > R:
            R = trailing_zeros

    #Add 2^R as an estimate for the number of distinct words seen       
    estimate = 2**R
    estimates.append(estimate)

    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))


Estimate on pass 1: 65536 distinct words
Estimate on pass 2: 32768 distinct words
Estimate on pass 3: 65536 distinct words
Estimate on pass 4: 65536 distinct words
Estimate on pass 5: 65536 distinct words
Estimate on pass 6: 16384 distinct words
Estimate on pass 7: 32768 distinct words
Estimate on pass 8: 65536 distinct words
Estimate on pass 9: 16384 distinct words
Estimate on pass 10: 65536 distinct words


In [56]:
#Compute the average and median
print("* Average of estimates: %.1f" % (statistics.mean(estimates)))
print("* Median  of estimates: %.1f" % (statistics.median(estimates)))

* Average of estimates: 18585.6
* Median  of estimates: 6144.0


In view of the heterogeneity of the estimates, the median is a better estimate. Since the extreme values did not represent reality accurately, the average could not represent it either.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>